<a href="https://colab.research.google.com/github/ClockWorkKid/Skin-Cancer-Detection-Segmentation/blob/main/notebooks/BioMed_Project_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doc Drive

In [ ]:
! nvidia-smi

Wed Nov 24 18:51:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Kaggle to Colab

In [ ]:
from google.colab import files 
data_to_load = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
import shutil, os
import cv2
import numpy as np

In [ ]:
!kaggle datasets download -d jessicali9530/lfw-dataset

 89% 100M/112M [00:01<00:00, 55.4MB/s] 
100% 112M/112M [00:01<00:00, 63.1MB/s]


In [ ]:
!unzip -q "/content/lfw-dataset.zip" -d /content/

In [ ]:
os.remove('/content/lfw-dataset.zip')

# Libraries

In [ ]:
%tensorflow_version 2.x
import numpy as np
import pandas as pd
import gc
import keras
import os
import shutil
import matplotlib.pyplot as plt
import cv2
import random

import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold

from skimage.transform import resize
import tensorflow as tf
import keras.backend as K
from keras.losses import binary_crossentropy

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import  ModelCheckpoint
from keras.layers import *
from keras.utils.vis_utils import plot_model
from keras import layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
from keras.callbacks import *
from keras.activations import sigmoid, tanh
import random
from keras.metrics import *
from sklearn.metrics import f1_score

!pip install tensorflow-addons

from PIL import Image
import tensorflow_addons as tfa 

     |████████████████████████████████| 1.1 MB 5.0 MB/s 


#Set Parameters

In [ ]:
img_dims= 256
batch_= 8
seed_ = 3+21+35+40+63
class_type = 'binary'
lr_ = 1e-4

seg_notclass= 0    #seg: 1, class: 0

#Metrics and Loss Functions

In [ ]:
def precision(y_true, y_pred): 
    """Precision metric. Only computes a batch-wise average of precision. Computes the precision, a metric for multi-label classification of how many selected items are relevant. 
-    """ 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) 
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1))) 
    precision = true_positives / (predicted_positives + K.epsilon()) 
    return precision 

def recall(y_true, y_pred): 
    """Recall metric. 
-    Only computes a batch-wise average of recall. Computes the recall, a metric for multi-label classification of how many relevant items are selected. 
-    """ 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) 
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1))) 
    recall = true_positives / (possible_positives + K.epsilon()) 
    return recall 

def fbeta_score(y_true, y_pred, beta=1.0): 

    """Computes the F score.  
-    The F score is the weighted harmonic mean of precision and recall. 
-    Here it is only computed as a batch-wise average, not globally. 
-    This is useful for multi-label classification, where input samples can be 
-    classified as sets of labels. By only using accuracy (precision) a model 
-    would achieve a perfect score by simply assigning every class to every 
-    input. In order to avoid this, a metric should penalize incorrect class 
-    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0) 
-    computes this, as a weighted mean of the proportion of correct class 
-    assignments vs. the proportion of incorrect class assignments.  
-    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning 
-    correct classes becomes more important, and with beta > 1 the metric is 
-    instead weighted towards penalizing incorrect class assignments. 
-    """ 
    if beta < 0: 
        raise ValueError('The lowest choosable beta is zero (only precision).') 

    # If there are no true positives, fix the F score at 0 like sklearn. 
    if K.sum(K.round(K.clip(y_true, 0.0, 1.0))) == 0: 

        return 0.0 
    p = precision(y_true, y_pred) 
    r = recall(y_true, y_pred) 
    bb = beta*beta
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon()) 
    return fbeta_score 


def dice_coef(y_true, y_pred):
    smooth=1
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    
    return (2.0 * intersection + smooth) / (union + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    return dice_coef_loss(y_true, y_pred) + bce(y_true, y_pred)

def iou(y_true, y_pred):
    smooth=1
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

def fmeasure(y_true, y_pred): 
    """Computes the f-measure, the harmonic mean of precision and recall. 
    Here it is only computed as a batch-wise average, not globally. 
    """ 
    return fbeta_score(y_true, y_pred, beta=1.0) 


#Models

## EfficientNet

In [ ]:
model = tf.keras.applications.EfficientNetB2(
    include_top=False,
    weights="imagenet",
    input_shape=(img_dims, img_dims, 3),
    pooling=None,
    classes=2,
)

# x = GlobalAveragePooling2D()(model.get_layer('pool4_conv').output) # DenseNet121
x = GlobalAveragePooling2D()(model.get_layer('block6a_expand_activation').output) #EffB2,B3, B4
# x = GlobalAveragePooling2D()(model.output)
x = Dense(512, 'relu')(x)
x = Dense(128, 'relu')(x)
if class_type=='binary':
  x = Dense(1, 'sigmoid')(x)
else:
  x = Dense(7, 'softmax')(x)

if not seg_notclass:
  model = Model(model.input, x)
  model.compile(loss= class_type+'_crossentropy',#'categorical_crossentropy', 
                optimizer= tf.keras.optimizers.Adam(learning_rate=lr_),
                metrics=['accuracy',
                         #tf.keras.metrics.Precision(thresholds=0.5, class_id= 3),
                         #tf.keras.metrics.Recall(thresholds=0.5, class_id= 3),
                         #tf.keras.metrics.AUC(num_thresholds=11)
                         precision,
                         recall,                         
                         fmeasure,
                        #  fbeta_score
                ])
  model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_3 (Rescaling)        (None, 256, 256, 3)  0           ['input_4[0][0]']                
                                                                                                  
 normalization_3 (Normalization  (None, 256, 256, 3)  7          ['rescaling_3[0][0]']            
 )                                                                                                
                                                                                            

##U-Net

In [ ]:
def unet(img_dims, start_neurons):

    input_layer = Input((img_dims, img_dims, 3))
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(convm)
    
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)

    model = Model(input_layer, output_layer)
    model.compile(loss= binary_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr_,
                                                     beta_1=0.9,
                                                     beta_2=0.999,
                                                     epsilon=None,
                                                     amsgrad=False),
                  metrics=[precision, recall, dice_coef, iou])
    
    return model


if seg_notclass:
  model = unet(img_dims, 16)
  model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_76 (Conv2D)             (None, 256, 256, 16  448         ['input_12[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_77 (Conv2D)             (None, 256, 256, 16  2320        ['conv2d_76[0][0]']              
                                )                                                           

# Data Generator

## For Classification

In [ ]:
# dir='.../Train'
# dir_val='.../Val'

dir = '/content/drive/MyDrive/Data/Skin Cancer Datasets/ISIC - M_v_B/Train' #From Jahin's drive
dir_val = '/content/drive/MyDrive/Data/Skin Cancer Datasets/ISIC - M_v_B/Val' #From Jahin's drive

# dir = '/content/drive/MyDrive/Biomed /Skin Cancer Datasets/HAM10000/Split/Train' #From Sayeed's drive
# dir_val = '/content/drive/MyDrive/Biomed /Skin Cancer Datasets/HAM10000/Split/Val' #From Sayeed's drive

datagen=ImageDataGenerator(rescale=1./255,
                            rotation_range=0.1,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            vertical_flip=True,
                            fill_mode='nearest')
train_gen = datagen.flow_from_directory(directory=dir,
                                                subset='training',
                                                target_size=(img_dims, img_dims), 
                                                color_mode = 'rgb',
                                                batch_size=batch_,
                                                class_mode=class_type,
                                                shuffle=True,
                                                seed= seed_)

datagen=ImageDataGenerator(rescale=1./255)

val_gen = datagen.flow_from_directory(directory=dir_val,
                                              subset='training',
                                              target_size=(img_dims, img_dims), 
                                              color_mode = 'rgb',
                                              batch_size=batch_,
                                              class_mode=class_type,
                                              shuffle=True,
                                              seed= seed_,)

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


## For Segmentation

In [ ]:
source_path = '/content/drive/MyDrive/Data/Skin Cancer Datasets/HAM10000/Split/Train'
source_path_val = '/content/drive/MyDrive/Data/Skin Cancer Datasets/HAM10000/Split/Val'


def adjust_data(img,mask):
    #.........
    img = img / 255.
    mask = mask / 255.
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    
    return (img, mask)

folders = ['Img','Mask']
folders_val=['Img', 'Mask']

def my_generator(batch_size_):
    imggen = ImageDataGenerator(rotation_range=0.1,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            vertical_flip=True,
                            fill_mode='nearest')
    
    data_generator = imggen.flow_from_directory(directory = source_path,
                                                target_size = (img_dims,img_dims),
                                                color_mode = 'rgb',
                                                classes = [folders[0]],
                                                class_mode = None,
                                                batch_size = batch_,
                                                seed = seed_,
                                                shuffle = True)
    
    mask_generator = imggen.flow_from_directory(directory = source_path,
                                                target_size = (img_dims,img_dims),
                                                 color_mode = 'grayscale',
                                                 classes = [folders[1]],
                                                class_mode = None,
                                                batch_size = batch_,
                                                seed = seed_,
                                                shuffle = True)
    
    train_gen = zip(data_generator, mask_generator)
    
    for (img, mask) in train_gen:
        img, mask = adjust_data(img, mask)
        yield (img,mask)
    
    # while True:
    #     x_batch = data_generator.next()
    #     y_batch = mask_generator.next()
    #     yield x_batch, y_batch





def my_generator_val(batch_size_):

    imggen = ImageDataGenerator()
    
    data_generator = imggen.flow_from_directory(directory = source_path_val,
                                                            target_size = (img_dims,img_dims),
                                                            color_mode = 'rgb',
                                                            classes = [folders_val[0]],
                                                            class_mode = None,
                                                            batch_size = batch_,
                                                            seed = seed_,
                                                            shuffle = False)
    mask_generator = imggen.flow_from_directory(directory = source_path_val,
                                                target_size = (img_dims,img_dims),
                                                color_mode = 'grayscale',
                                                classes = [folders_val[1]],
                                                class_mode = None,
                                                batch_size = batch_,
                                                seed = seed_,
                                                shuffle = False)
    # while True:
    #     x_batch = data_generator.next()
    #     y_batch = mask_generator.next()
    #     yield x_batch, y_batch
    train_gen = zip(data_generator, mask_generator)
    
    for (img, mask) in train_gen:
        img, mask = adjust_data(img, mask)
        yield (img,mask)

In [ ]:
len(os.listdir(source_path_val+'/'+folders_val[1])), len(os.listdir(source_path_val+'/'+folders_val[0])), len(os.listdir(source_path+'/'+folders[1])), len(os.listdir(source_path+'/'+folders[0]))

(1002, 1002, 8012, 8012)

#Training

In [ ]:
# model.load_weights('/content/drive/MyDrive/Data/Skin Cancer Datasets/Weights/MvB_EffB2_truncated.h5')

weight_saver = ModelCheckpoint('/content/drive/MyDrive/Data/Skin Cancer Datasets/Weights/MvB_EffB2_truncated.h5', 
                               monitor='val_accuracy',
                               mode ='max',
                               save_best_only=True,
                               save_weights_only=True,
                               verbose=1)

annealer = ReduceLROnPlateau(monitor='loss',
                             factor=0.5,
                             patience=5,
                             min_lr=1e-8,
                             verbose=1)

##>>> *** For Classification
history= model.fit(train_gen,
                      steps_per_epoch=2637//(batch_),
                      epochs=100,
                      verbose=1,
                      validation_data=val_gen,
                      validation_steps=660//batch_, 
                      callbacks=[annealer, weight_saver])


##>>> *** For Segmentation
# history= model.fit(my_generator(batch_),
#          steps_per_epoch = len(os.listdir(source_path+'/'+folders[1]))//batch_,
#          validation_data = my_generator_val(batch_), 
#          validation_steps= len(os.listdir(source_path_val+'/'+folders_val[1]))//batch_,                          
#          epochs=200,
#          verbose=1, 
#          callbacks = [weight_saver , annealer]
#          )

Epoch 1/100


KeyboardInterrupt: ignored

In [ ]:
model.load_weights('/content/drive/MyDrive/Data/Skin Cancer Datasets/Weights/MvB_EffB2_truncated.h5')


# metrics=['accuracy',  precision,  recall,  fmeasure, ])

model.evaluate(val_gen, verbose=1, steps=1002//8)

125/125 [==============================] - 57s 452ms/step - loss: 0.2357 - accuracy: 0.8848 - precision: 0.8952 - recall: 0.8568 - fmeasure: 0.8551


[0.23572050034999847,
 0.8848484754562378,
 0.8951519727706909,
 0.856827437877655,
 0.8550757765769958]